In [0]:
#importing libraries
import sys
import pandas as pd
import os

In [0]:
#reading the dataset into a Dataframe
df=pd.read_csv('Complete_Data.csv')

In [4]:
#View the first 5 rows
df.head()

,text_reviews_count,country_code,language_code,popular_shelves,is_ebook,average_rating,description,authors,publication_year,book_id,ratings_count,title,title_without_series,user_id,review_id,is_read,rating,date_added,date_updated,read_at,started_at,review_text,n_votes,n_comments,authorid
0,1,US,NaN,"[{'count': '228', 'name': 'to-read'}, {'count'...",True,4.12,Lillian Ann Cross is forced to live the worst ...,"[{'author_id': '8551671', 'role': ''}]",NaN,25742454,1,The Switchblade Mamma,The Switchblade Mamma,f1830550c10246b9924d03fbb7858a3b,cadab2ad2d162553b3b6b2a5884ec0ea,True,5,Wed Jun 15 16:38:58 -0700 2016,Wed Jun 15 16:38:59 -0700 2016,Wed Jun 15 16:38:59 -0700 2016,NaN,Raw and amazing \n This book is unlike any oth...,0,0,['8551671']
1,2,US,fre,"[{'count': '2', 'name': 'bd'}, {'count': '2', ...",False,3.94,"Florence Dupre Latour raconte comment, de son ...","[{'author_id': '3274315', 'role': ''}]",2016.0,30128855,16,Cruelle,Cruelle,4867af4dbac45448491c46b43363f240,6133733899c4a1aa8885b3be42e03c22,True,4,Sun Aug 14 09:49:50 -0700 2016,Sun Sep 04 11:30:18 -0700 2016,Sun Sep 04 11:47:36 -0700 2016,NaN,"Un livre troublant, une oeuvre a decouvrir.",2,0,['3274315']
2,5,US,eng,"[{'count': '493', 'name': 'to-read'}, {'count'...",False,4.28,The questions plaguing Captain America's dream...,"[{'author_id': '37450', 'role': ''}]",2012.0,13571772,51,Captain America: Winter Soldier (The Ultimate ...,Captain America: Winter Soldier (The Ultimate ...,568ce0dddc00192a814476ab5441acbb,319585c91e4c766627ebc4ebae533e76,True,3,Thu Sep 11 05:22:01 -0700 2014,Thu Sep 11 05:22:10 -0700 2014,Thu Sep 11 05:22:10 -0700 2014,NaN,It's an o.k. story. Much better than some othe...,0,0,['37450']
3,5,US,eng,"[{'count': '493', 'name': 'to-read'}, {'count'...",False,4.28,The questions plaguing Captain America's dream...,"[{'author_id': '37450', 'role': ''}]",2012.0,13571772,51,Captain America: Winter Soldier (The Ultimate ...,Captain America: Winter Soldier (The Ultimate ...,ae681bb05174eaa3d2124d51262b76ad,2fba5f2a163214c96cf01c05c404e142,True,3,Mon Oct 08 05:05:03 -0700 2012,Sat Aug 10 05:52:40 -0700 2013,Wed Oct 10 05:56:04 -0700 2012,Mon Oct 08 00:00:00 -0700 2012,A Captain America story involving a Russian as...,0,0,['37450']
4,5,US,eng,"[{'count': '493', 'name': 'to-read'}, {'count'...",False,4.28,The questions plaguing Captain America's dream...,"[{'author_id': '37450', 'role': ''}]",2012.0,13571772,51,Captain America: Winter Soldier (The Ultimate ...,Captain America: Winter Soldier (The Ultimate ...,396fe0f07a53ad6be40e232c28a35e18,76c1244b6141bbe238d5385d9a19777b,True,4,Wed Nov 21 18:49:31 -0800 2012,Wed Nov 21 18:52:49 -0800 2012,Thu Nov 22 18:52:49 -0800 2012,NaN,I read this in order to prepare myself for the...,0,0,['37450']


In [5]:
#Get the dataset info on number of rows and columns and each column information
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 542338 entries, 0 to 542337
Data columns (total 25 columns):
text_reviews_count      542338 non-null int64
country_code            542338 non-null object
language_code           440941 non-null object
popular_shelves         542338 non-null object
is_ebook                542338 non-null bool
average_rating          542338 non-null float64
description             503862 non-null object
authors                 542338 non-null object
publication_year        480268 non-null float64
book_id                 542338 non-null int64
ratings_count           542338 non-null int64
title                   542338 non-null object
title_without_series    542338 non-null object
user_id                 542338 non-null object
review_id               542338 non-null object
is_read                 542338 non-null bool
rating                  542338 non-null int64
date_added              542338 non-null object
date_updated            542338 non-null object
re

In [0]:
#take only the required columns and store it in another dataset
content_data = df[['user_id', 'book_id', 'country_code', 'language_code', 'rating', 'authorid', 'description', 'title']]

In [7]:
content_data.head()

,user_id,book_id,country_code,language_code,rating,authorid,description,title
0,f1830550c10246b9924d03fbb7858a3b,25742454,US,NaN,5,['8551671'],Lillian Ann Cross is forced to live the worst ...,The Switchblade Mamma
1,4867af4dbac45448491c46b43363f240,30128855,US,fre,4,['3274315'],"Florence Dupre Latour raconte comment, de son ...",Cruelle
2,568ce0dddc00192a814476ab5441acbb,13571772,US,eng,3,['37450'],The questions plaguing Captain America's dream...,Captain America: Winter Soldier (The Ultimate ...
3,ae681bb05174eaa3d2124d51262b76ad,13571772,US,eng,3,['37450'],The questions plaguing Captain America's dream...,Captain America: Winter Soldier (The Ultimate ...
4,396fe0f07a53ad6be40e232c28a35e18,13571772,US,eng,4,['37450'],The questions plaguing Captain America's dream...,Captain America: Winter Soldier (The Ultimate ...


In [8]:
content_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 542338 entries, 0 to 542337
Data columns (total 8 columns):
user_id          542338 non-null object
book_id          542338 non-null int64
country_code     542338 non-null object
language_code    440941 non-null object
rating           542338 non-null int64
authorid         542338 non-null object
description      503862 non-null object
title            542338 non-null object
dtypes: int64(2), object(6)
memory usage: 33.1+ MB


In [0]:
#Taking only the book metadata as we are doing cntent based and we do not need user information
book_data = content_data[['book_id', 'description', 'title', 'country_code', 'language_code', 'authorid']]

In [10]:
book_data

,book_id,description,title,country_code,language_code,authorid
0,25742454,Lillian Ann Cross is forced to live the worst ...,The Switchblade Mamma,US,NaN,['8551671']
1,30128855,"Florence Dupre Latour raconte comment, de son ...",Cruelle,US,fre,['3274315']
2,13571772,The questions plaguing Captain America's dream...,Captain America: Winter Soldier (The Ultimate ...,US,eng,['37450']
3,13571772,The questions plaguing Captain America's dream...,Captain America: Winter Soldier (The Ultimate ...,US,eng,['37450']
4,13571772,The questions plaguing Captain America's dream...,Captain America: Winter Soldier (The Ultimate ...,US,eng,['37450']
...,...,...,...,...,...,...
542333,20653416,EL GRAN CLASICO DEL GENERO FANTASTICO EN UN VO...,La búsqueda del pájaro del tiempo,US,spa,"['665253', '658602']"
542334,28928706,Prepare yourself for Dejah Thoris as you've ne...,Dejah Thoris #1,US,eng,"['5740317', '8252667']"
542335,28928706,Prepare yourself for Dejah Thoris as you've ne...,Dejah Thoris #1,US,eng,"['5740317', '8252667']"
542336,23363286,"In the city that never sleeps, spirits are equ...",Dead Endings,US,eng,"['7741497', '761035']"


In [0]:
#Since we have duplicate bookids, we do a groupby on bookid
book_data = book_data.groupby(['book_id']).agg(lambda x: list(x)).reset_index()

In [0]:
#We are getting same values for title, description, country code, language code after groupby as it is the same bookid. So we take only one value for each
book_data['title'] = book_data['title'].apply(lambda x: x[:1])
book_data['description'] = book_data['description'].apply(lambda x: x[:1])
book_data['country_code'] = book_data['country_code'].apply(lambda x: x[:1])
book_data['language_code'] = book_data['language_code'].apply(lambda x: x[:1])


In [13]:
#Viewing the first 5 records of book_data
book_data.head()

,book_id,description,title,country_code,language_code,authorid
0,374,[Beautifully illustrated and deeply imaginativ...,[Salmon Doubts],[US],[nan],"[['213'], ['213'], ['213'], ['213'], ['213'], ..."
1,450,[I cannot possibly offer enough praise for Dav...,[Neon Lit: Paul Auster's City of Glass],[US],[eng],"[['28996', '28995', '296961', '28997'], ['2899..."
2,586,"[Authoritative, eye-popping, and massive, Art ...",[Art of Modern Rock: The Poster Explosion],[US],[nan],"[['364', '94433', '3849785', '5675001']]"
3,649,[Logo Design Workbookfocuses on creating power...,[Logo Design Workbook: A Hands-On Guide to Cre...,[US],[nan],"[['295258', '3849823', '3446002', '3849824'], ..."
4,690,[nan],[The End of Print],[US],[nan],"[['454'], ['454']]"


In [14]:
book_data

,book_id,description,title,country_code,language_code,authorid
0,374,[Beautifully illustrated and deeply imaginativ...,[Salmon Doubts],[US],[nan],"[['213'], ['213'], ['213'], ['213'], ['213'], ..."
1,450,[I cannot possibly offer enough praise for Dav...,[Neon Lit: Paul Auster's City of Glass],[US],[eng],"[['28996', '28995', '296961', '28997'], ['2899..."
2,586,"[Authoritative, eye-popping, and massive, Art ...",[Art of Modern Rock: The Poster Explosion],[US],[nan],"[['364', '94433', '3849785', '5675001']]"
3,649,[Logo Design Workbookfocuses on creating power...,[Logo Design Workbook: A Hands-On Guide to Cre...,[US],[nan],"[['295258', '3849823', '3446002', '3849824'], ..."
4,690,[nan],[The End of Print],[US],[nan],"[['454'], ['454']]"
...,...,...,...,...,...,...
89306,36478518,[nan],[Star Wars: Jedi of the Republic - Mace Windu ...,[US],[nan],"[['7819976', '493715', '507122']]"
89307,36480175,[Le monde est en danger. Probleme : ils sont l...,[Juste un peu de cendres],[US],[fre],"[['221441', '4081971']]"
89308,36493971,[nan],"[Cardcaptor Sakura, Omnibus 3]",[US],[eng],[['8476']]
89309,36494299,[The conclusion of the Cardcaptor Sakura saga ...,"[Cardcaptor Sakura, Omnibus 4]",[US],[eng],[['8476']]


In [0]:
#We need to combine the columns description, authorid, country code and language code to apply tf-idf on these columns as a whole. We have to convert it to string as we cannot combine on list
book_data = book_data.applymap(str).reset_index()

In [0]:
#Function that creates a soup out of the desired metadata
def create_soup(x):
    return ' '.join(x['description']) + ' ' + ' '.join(x['authorid']) + ' ' + x['country_code'] + ' ' + ' '.join(x['language_code'])

In [0]:
#We are ceating a soup out the desired metadata and storing it in another column named soup
book_data['soup'] = book_data.apply(create_soup, axis=1)

In [0]:
#book_data = book_data.head(1000)

In [19]:
#Import TfIdfVectorizer from the scikit-learn library
from sklearn.feature_extraction.text import TfidfVectorizer

#Define a TF-IDF Vectorizer Object
tf = TfidfVectorizer(token_pattern=r"(?u)\b\w+\b", stop_words='english')

#Construct the required TF-IDF matrix by applying the fit_transform method on the overview feature
tfidf_matrix = tf.fit_transform(book_data['soup'])

#Output the shape of tfidf_matrix
tfidf_matrix.shape

(1000, 34)

In [0]:
# Import cosine_similarity function
from sklearn.metrics.pairwise import cosine_similarity

# Compute the cosine similarity matrix
cosine_sim = cosine_similarity(tfidf_matrix, tfidf_matrix)

In [0]:
#Construct a reverse mapping of indices and book titles, and drop duplicate titles, if any
indices = pd.Series(book_data.index, index=book_data['title']).drop_duplicates()

In [0]:
# Function that takes in book title as input and gives recommendations 
def content_recommender(title, cosine_sim=cosine_sim, df=book_data, indices=indices):
    # Obtain the index of the book that matches the title
    idx = indices[title]

    # Get the pairwsie similarity scores of all books with that book
    # And convert it into a list of tuples as described above
    sim_scores = list(enumerate(cosine_sim[idx]))

    # Sort the books based on the cosine similarity scores
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)

    # Get the scores of the 10 most similar books. Ignore the first book.
    sim_scores = sim_scores[1:11]

    # Get the book indices
    book_indices = [i[0] for i in sim_scores]

    # Return the top 10 most similar books
    # return df['title'].iloc[book_indices]
    book =[]
    score = []
    index=[]
    book = df['title'].iloc[book_indices]
    index,score = map(list,zip(*sim_scores))
    #store book tiltes and similarity scores in a data frame
    recommend_metadata=pd.DataFrame({'book' : book, 'Score' : score})
    print(recommend_metadata)
    #result1 = recommend_metadata
    recommend_metadata.to_csv("result1.csv", index=False)
    #return df['title'].iloc[book_indices]

In [23]:
#Get recommendations for Emma, Vol. 01
content_recommender("['Emma, Vol. 01']")

                                                  book     Score
875                                   ['Black Orchid']  0.968167
233  ['The Tragical Comedy or Comical Tragedy of Mr...  0.948917
377                                  ['Emma, Vol. 02']  0.934145
450                     ['The Wake (The Sandman #10)']  0.916117
256                             ['The Books of Magic']  0.904059
876                    ["Neil Gaiman's Midnight Days"]  0.899404
448                 ['A Game of You (The Sandman #5)']  0.895435
748  ["Kristy's Great Idea (Baby-Sitters Club Graph...  0.890777
288                   ['Death: The Time of Your Life']  0.886756
495    ['Revolutionary Girl Utena, Vol. 3: To Sprout']  0.884868


In [24]:
#Get recommendations for Black Orchid
content_recommender("['Black Orchid']")

                                                  book     Score
233  ['The Tragical Comedy or Comical Tragedy of Mr...  0.981432
376                                  ['Emma, Vol. 01']  0.968167
377                                  ['Emma, Vol. 02']  0.945667
288                   ['Death: The Time of Your Life']  0.937875
428               ['The Absolute Sandman, Volume One']  0.901476
256                             ['The Books of Magic']  0.896892
445              ["The Doll's House (The Sandman #2)"]  0.894554
903                             ['Bring Me Your Love']  0.891053
876                    ["Neil Gaiman's Midnight Days"]  0.890316
450                     ['The Wake (The Sandman #10)']  0.886178


In [25]:
#Get recommendations for Inu Yasha Animanga, Volume 17
content_recommender("['Inu Yasha Animanga, Volume 17']")

                                                  book     Score
359                  ['Inu Yasha Animanga, Volume 17']  1.000000
360                    ['Inuyasha Ani-Manga, Vol. 18']  1.000000
362                    ['Inuyasha Ani-Manga, Vol. 21']  1.000000
361  ["InuYasha: Naraku's Perfect New Form (InuYash...  0.992426
736  ['InuYasha: The Rebirth of Naraku (InuYasha, #...  0.991212
596                         ['Dokebi Bride, Volume 1']  0.985645
775  ['German Modern: Graphic Design from Wilhelm t...  0.981842
634     ['Ranma 1/2, Vol. 27 (Ranma ½ (US 2nd), #27)']  0.981542
861          ['Mortelles En Tête (Grand Vampire, #2)']  0.980216
619                    ['X/1999, Volume 05: Serenade']  0.979540


In [0]:
#Get all the records who has the title "Inu Yasha Animanga, Volume 17"
y = df[df['title'].astype(str).str.contains("Inu Yasha Animanga, Volume 17")]

In [27]:
y

,text_reviews_count,country_code,language_code,popular_shelves,is_ebook,average_rating,description,authors,publication_year,book_id,ratings_count,title,title_without_series,user_id,review_id,is_read,rating,date_added,date_updated,read_at,started_at,review_text,n_votes,n_comments,authorid
532638,3,US,NaN,"[{'count': '78', 'name': 'to-read'}, {'count':...",False,4.26,"Pulled back in time to Japan's ancient past, J...","[{'author_id': '12948', 'role': ''}]",2006.0,22706,141,"Inu Yasha Animanga, Volume 17","Inu Yasha Animanga, Volume 17",1bacb7b243952eb6c923563854efa141,842ff46e0770ef0cc81a24d4b9aff212,False,0,Mon Jul 05 14:05:47 -0700 2010,Mon Jul 05 14:05:56 -0700 2010,NaN,NaN,F TAK ani-manga v.17,0,0,['12948']
532639,3,US,NaN,"[{'count': '78', 'name': 'to-read'}, {'count':...",False,4.26,"Pulled back in time to Japan's ancient past, J...","[{'author_id': '12948', 'role': ''}]",2006.0,22706,141,"Inu Yasha Animanga, Volume 17","Inu Yasha Animanga, Volume 17",babc631ce1f8d2deab56f65cee762062,42116271e5ce146b9a1932f8b950dd19,True,4,Mon Feb 09 12:34:28 -0800 2015,Sat Oct 17 21:14:43 -0700 2015,Sun Jan 01 00:00:00 -0800 2006,NaN,What first drew me to this series was the roma...,0,0,['12948']


In [0]:
#Pick a user who has read Inu Yasha Animanga, Volume 17 and get the list of all his books just to compare his list contains some of the recommendd books for Inu Yasha Animanga, Volume 17
df[df['user_id']=='babc631ce1f8d2deab56f65cee762062']['title'].values

array(['Blood+, Vol. 03 (Blood+, #3)', 'Blood+, Vol. 02 (Blood+, #2)',
       'Honey & Honey Drops #6', 'Red River, Vol. 17 (Red River, #17)',
       'Boys Over Flowers: Hana Yori Dango, Vol. 13 (Boys Over Flowers, #13)',
       'Claymore, Vol. 12: The Souls of the Fallen (Claymore, #12)',
       'Itazura Na Kiss, Volume 1 (Itazura Na Kiss, #1)',
       'InuYasha: The Struggle Continues (InuYasha, #39)',
       'Love Berrish!, Vol. 02', 'Inuyasha, Volume 09',
       'Princess Jellyfish, Tome 2', 'Skip Beat!, Vol. 22',
       'Nana, Vol. 6 (Nana, #6)', 'Yu Yu Hakusho, Volume 19',
       'Yu Yu Hakusho, Volume 1: Goodbye, Material World! (Yu Yu Hakusho, #1)',
       'Inuyasha, Volume 15', 'InuYasha: Mirror Image (InuYasha, #16)',
       'Nana, Vol. 13 (Nana, #13)', 'Nana, Vol. 14 (Nana, #14)',
       'Nana 15', 'Nana 16 (Nana, #16)',
       'Death Note, Vol. 11: Kindred Spirits (Death Note, #11)',
       'Death Note, Vol. 12: Finis (Death Note, #12)',
       'Boys Over Flowers: Hana Yori

In [0]:
#store the list of books read by user in a variable
result2 = df[df['user_id']=='babc631ce1f8d2deab56f65cee762062']['title'].values

In [0]:
#convert to a data frame
result0 = pd.DataFrame(result2)

In [0]:
result0['book'] = result0

In [31]:
result0['book']

0                           Blood+, Vol. 03 (Blood+, #3)
1                           Blood+, Vol. 02 (Blood+, #2)
2                                 Honey & Honey Drops #6
3                    Red River, Vol. 17 (Red River, #17)
4      Boys Over Flowers: Hana Yori Dango, Vol. 13 (B...
                             ...                        
747    InuYasha: Liars and Ogres and Monkeys...Oh, My...
748                              La Corda d'Oro, Vol. 14
749    Peach Girl: Change of Heart, Vol. 4 (Peach Gir...
750      Path of Revenge (W.I.T.C.H. Chapter Books, #16)
751         Haruka: Beyond the Stream of Time, Volume 16
Name: book, Length: 752, dtype: object

In [0]:
#get the list of recommended book for the book Inu Yasha Animanga in a data frame as it is saved in result1.csv
result1 = df=pd.read_csv('/content/result1.csv')

In [33]:
result1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10 entries, 0 to 9
Data columns (total 2 columns):
book     10 non-null object
Score    10 non-null float64
dtypes: float64(1), object(1)
memory usage: 288.0+ bytes


In [0]:
result1['book'] = result1['book'].str[1:-1]


In [35]:
result1['book']

0                      'Inu Yasha Animanga, Volume 17'
1                        'Inuyasha Ani-Manga, Vol. 18'
2                        'Inuyasha Ani-Manga, Vol. 21'
3    "InuYasha: Naraku's Perfect New Form (InuYasha...
4    'InuYasha: The Rebirth of Naraku (InuYasha, #28)'
5                             'Dokebi Bride, Volume 1'
6    'German Modern: Graphic Design from Wilhelm to...
7         'Ranma 1/2, Vol. 27 (Ranma ½ (US 2nd), #27)'
8              'Mortelles En Tête (Grand Vampire, #2)'
9                        'X/1999, Volume 05: Serenade'
Name: book, dtype: object

In [0]:
#Check if the any of the books recommnded are there in the user list
d1=result1['book'].isin(result0['book'])